In [1]:
!pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
import random
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time

In [2]:
df_links=pd.read_excel("ConvocatoriasPublicas.xlsx")
df_links

,CodigoConvocatoria,CodigoSicAgente,NombreAgenteComprador,FechaPublicacionAviso,PeriodoAContratar,FechaInicio,FechaFin,DuracionDias,meses,union,...,FechaLimiteRecepcionOfertas,AudienciaPublica,FechaMaximaFormalizacionContratos,Estado,TipoMercado,ExclusivaFNCER,FechaSuspension,FechaReactivacion,FechaCancelacion,FechaCierreConvocatoria
0,CP-EMIC2024-002,EMIC,EMCALI,31/05/2024,01/01/2025 al 31/12/2028,01/01/2025,31/12/2028,1460,48.000000,CP-EMIC2024-002/EMCALI/EMIC/01%2F01%2F2025/31%...,...,28/06/2024,12/07/2024,30/07/2024,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,08/08/2024
1,CP-CSSC2024-004,CSSC,Air-e,30/05/2024,01/08/2024 al 31/12/2037,01/08/2024,31/12/2037,4900,161.095890,CP-CSSC2024-004/Air-e/CSSC/01%2F08%2F2024/31%2...,...,21/06/2024,05/07/2024,17/07/2024,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,25/07/2024
2,CP-EBSC2024-003,EBSC,EBSA,21/05/2024,01/01/2025 al 31/12/2039,01/01/2025,31/12/2039,5477,180.065753,CP-EBSC2024-003/EBSA/EBSC/01%2F01%2F2025/31%2F...,...,18/06/2024,02/07/2024,19/07/2024,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,22/07/2024
3,CP-RTQC2024-002,RTQC,RUITOQUE,07/05/2024,01/08/2024 al 31/12/2028,01/08/2024,31/12/2028,1613,53.030137,CP-RTQC2024-002/RUITOQUE/RTQC/01%2F08%2F2024/3...,...,04/06/2024,18/06/2024,04/07/2024,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,09/07/2024
4,CP-BIAC2024-001,BIAC,BIAE,02/05/2024,01/01/2025 al 31/12/2029,01/01/2025,31/12/2029,1825,60.000000,CP-BIAC2024-001/BIAE/BIAC/01%2F01%2F2025/31%2F...,...,29/05/2024,13/06/2024,27/06/2024,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,10/07/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,CP-PEEC2020-002,PEEC,PEESA,03/03/2020,01/06/2020 al 31/12/2022,01/06/2020,31/12/2022,943,31.002740,CP-PEEC2020-002/PEESA/PEEC/01%2F06%2F2020/31%2...,...,27/04/2020,09/05/2020,29/05/2020,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,03/06/2020
239,CP-GNCC2020-001,GNCC,VATIA S.A.,20/02/2020,01/08/2020 al 31/12/2025,01/08/2020,31/12/2025,1978,65.030137,CP-GNCC2020-001/VATIA S.A./GNCC/01%2F08%2F2020...,...,29/04/2020,13/05/2020,01/06/2020,Cerrada y adjudicada,Regulado,NO,14/04/2020,NaN,NaN,04/06/2020
240,CP-EDIC2020-001,EDIC,ELECTRICARIBE.,11/02/2020,15/05/2020 al 31/12/2025,15/05/2020,31/12/2025,2056,67.594521,CP-EDIC2020-001/ELECTRICARIBE./EDIC/15%2F05%2F...,...,31/03/2020,07/05/2020,29/05/2020,Cerrada y adjudicada,Regulado,NO,13/04/2020,NaN,NaN,01/06/2020
241,CP-EPIC2020-001,EPIC,CELSIA TOLIMA,03/02/2020,01/05/2020 al 31/12/2024,01/05/2020,31/12/2024,1705,56.054795,CP-EPIC2020-001/CELSIA TOLIMA/EPIC/01%2F05%2F2...,...,24/03/2020,03/04/2020,27/04/2020,Cerrada y adjudicada,Regulado,NO,NaN,NaN,NaN,28/04/2020


In [5]:
def scraping(link):
    #Ubicacion de webdrive
    ubicacion = "chromedriver" #Ruta del driver
    driver = webdriver.Chrome(ubicacion)

    #Definimos link
    home_link = "https://sicep.xm.com.co/public-announcements/electronic-file/"
    search_kw = str(link)
    driver.get(home_link+search_kw)

    #abrimos pagina
    page = BeautifulSoup(driver.page_source,"html.parser")
    #tiempo de espera
    time.sleep(7)
    # Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
    try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
        disclaimer1 = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[1]')
        disclaimer1.click() # lo obtenemos y le damos click
    except Exception as e:
        print (e) 
        None
    
    #tiempo de espera
    time.sleep(4)
    
    #buscamos fecha inicio
    periodo_inicio = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[1]/p[2]')
    periodo_inicio = periodo_inicio.text.strip()

    #buscamos fecha final
    periodo_fin = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[2]/p[2]')
    periodo_fin = periodo_fin.text.strip()

    #descripcion
    descripcion = driver.find_element('xpath', '//*[@id="detConvocatoria"]/div[5]/div/p')
    descripcion = descripcion.text.strip()

    #precios
    # Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
    try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
        disclaimer = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[7]')
        disclaimer.click() # lo obtenemos y le damos click
    except Exception as e:
        print (e) 
    None
    # Esperar 2 segundos
    time.sleep(4)
    # Encontrar el elemento que contiene la tabla y obtener su contenido HTML
    tabla_elements = driver.find_elements('xpath','//*[@id="ui-accordiontab-6-content"]/div/app-publication-information/div/div[2]/div[3]/p-table/div/div')
    tabla_html_list = []
    for tabla_element in tabla_elements:
        tabla_html = tabla_element.get_attribute('innerHTML')
        tabla_html_list.append(tabla_html)
    # Utilizar BeautifulSoup para analizar el contenido HTML y extraer la información que necesitas
    soup = BeautifulSoup(tabla_html, 'html.parser')
    tabla = soup.find('table')
    # Imprimir la tabla
    tabla_html = tabla.prettify()

    # Convertir la tabla HTML en un DataFrame
    import pandas as pd
    df = pd.read_html(tabla_html)[0]
    df["periodo_inicio"]=periodo_inicio
    df["periodo_fin"]=periodo_fin
    df["descripcion"]=descripcion
    df["union"]=link
    time.sleep(1)
    driver.quit()
    return df

Creamos resultado

In [8]:
link=df_links.union[0]
df_resultado = scraping(link)
#df_resultado = df_links.iloc[[0]].merge(df_resultado, on='union')
df_resultado

,Productos,Cantidad de energía demandada GWh,Cantidad de energía adjudicada GWh,Precio promedio ponderado adjudicado ($/KWh),periodo_inicio,periodo_fin,descripcion,union
0,CP-EMIC2024-002-PROD01,NaN,NaN,NaN,01/01/2025,31/12/2028,EMCALI requiere contratar el suministro de ene...,CP-EMIC2024-002/EMCALI/EMIC/01%2F01%2F2025/31%...
1,CP-EMIC2024-002-PROD02,1034.0,206.0,324.0,01/01/2025,31/12/2028,EMCALI requiere contratar el suministro de ene...,CP-EMIC2024-002/EMCALI/EMIC/01%2F01%2F2025/31%...
2,CP-EMIC2024-002-PROD03,NaN,NaN,NaN,01/01/2025,31/12/2028,EMCALI requiere contratar el suministro de ene...,CP-EMIC2024-002/EMCALI/EMIC/01%2F01%2F2025/31%...


In [10]:
for i in range(1,243):
    print(i)
    link=df_links.union[i]
    df_scr = scraping(link)
    df_scr = df_links.iloc[[i]].merge(df_scr, on='union')

    #concadenamos al resultado
    df_resultado = pd.concat([df_resultado, df_scr], ignore_index=True)

    #tiempo de espera
    time.sleep(8)

    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="expediente"]/div/p-accordiontab[1]"}
  (Session info: chrome=128.0.6613.85)
Stacktrace:
	

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="titleperiodtohire"]/div[1]/p[2]"}
  (Session info: chrome=128.0.6613.85)
Stacktrace:
	GetHandleVerifier [0x00007FF6EAAEB632+29090]
	(No symbol) [0x00007FF6EAA5E6E9]
	(No symbol) [0x00007FF6EA91B1CA]
	(No symbol) [0x00007FF6EA96EFD7]
	(No symbol) [0x00007FF6EA96F22C]
	(No symbol) [0x00007FF6EA9B97F7]
	(No symbol) [0x00007FF6EA99672F]
	(No symbol) [0x00007FF6EA9B65D9]
	(No symbol) [0x00007FF6EA996493]
	(No symbol) [0x00007FF6EA9609B1]
	(No symbol) [0x00007FF6EA961B11]
	GetHandleVerifier [0x00007FF6EAE0881D+3294093]
	GetHandleVerifier [0x00007FF6EAE54403+3604339]
	GetHandleVerifier [0x00007FF6EAE4A2C7+3563063]
	GetHandleVerifier [0x00007FF6EABA6F16+797318]
	(No symbol) [0x00007FF6EAA6986F]
	(No symbol) [0x00007FF6EAA65454]
	(No symbol) [0x00007FF6EAA655E0]
	(No symbol) [0x00007FF6EAA54A7F]
	BaseThreadInitThunk [0x00007FFB6E3D257D+29]
	RtlUserThreadStart [0x00007FFB706EAF28+40

In [11]:
df_resultado.to_csv("dfssdfds.csv")

In [13]:
df_resultado.to_excel("sadada.xls")

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_25352\2518002722.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_resultado.to_excel("sadada.xls")


In [115]:
ubicacion = "D:\\Descargas\\chromedriver_win32\\chromedriver" #Ruta del driver
driver = webdriver.Chrome(ubicacion)

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\557961577.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion)


In [116]:
home_link = "https://sicep.xm.com.co/public-announcements/electronic-file/"
search_kw = str(df_links.union[1])

driver.get(home_link+search_kw)

page = BeautifulSoup(driver.page_source,"html.parser")

#bloque = page.find('li', attrs={'class':'s-item', 'data-view':True})

In [117]:
# Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
  disclaimer1 = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[1]')
  disclaimer1.click() # lo obtenemos y le damos click
except Exception as e:
  print (e) 
  None

In [118]:
periodo_inicio = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[1]/p[2]')
periodo_inicio = periodo_inicio.text.strip()
periodo_inicio

'01/02/2023'

In [119]:
periodo_fin = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[2]/p[2]')
periodo_fin = periodo_fin.text.strip()
periodo_fin

'31/12/2026'

In [120]:
descipcion = driver.find_element('xpath', '//*[@id="detConvocatoria"]/div[5]/div/p')
descipcion = descipcion.text.strip()
descipcion

'VATIA S.A E.S.P. -VATIA- está interesada en recibir ofertas para la contratación del suministro de energía eléctrica con destino a atender la demanda del mercado regulado en el periodo comprendido desde las 00:00 horas de la fecha de registro hasta las 24:00 del día 31 de diciembre de 2026.'

In [82]:
import time

In [121]:
# Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
  disclaimer = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[7]')
  disclaimer.click() # lo obtenemos y le damos click
except Exception as e:
  print (e) 
  None
# Esperar 2 segundos
time.sleep(2)
# Encontrar el elemento que contiene la tabla y obtener su contenido HTML
tabla_elements = driver.find_elements('xpath','//*[@id="ui-accordiontab-6-content"]/div/app-publication-information/div/div[2]/div[3]/p-table/div/div')
tabla_html_list = []
for tabla_element in tabla_elements:
    tabla_html = tabla_element.get_attribute('innerHTML')
    tabla_html_list.append(tabla_html)
# Utilizar BeautifulSoup para analizar el contenido HTML y extraer la información que necesitas
soup = BeautifulSoup(tabla_html, 'html.parser')
tabla = soup.find('table')
# Imprimir la tabla
tabla_html = tabla.prettify()

In [122]:
# Convertir la tabla HTML en un DataFrame
import pandas as pd
df = pd.read_html(tabla_html)[0]
df

,Productos,Cantidad de energía demandada GWh,Cantidad de energía adjudicada GWh,Precio promedio ponderado adjudicado ($/KWh)
0,CP-GNCC2022-003-PROD01,115.00,108.75,276.14
1,CP-GNCC2022-003-PROD02,235.60,235.60,281.90
2,CP-GNCC2022-003-PROD03,135.55,0.00,0.00
3,CP-GNCC2022-003-PROD04,161.42,0.00,0.00


In [92]:
driver.quit()

**DATOS AUTOGENERACION

In [93]:
!pip install pydataxm 
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar
import datetime as dt  
objetoAPI = pydataxm.ReadDB()   

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/52.1 kB ? eta -:--:--
     ---------------------------------------- 52.1/52.1 kB ? eta 0:00:00
  Created wheel for pydataxm: filename=pydataxm-0.3.2-py3-none-any.whl size=3493 sha256=d593edd42fb9a2d688577dd62cfb2e1914bddd9ffaeca1bf28fa9dc18964d9cc
  Stored in directory: c:\users\dellphoto\appdata\local\pip\cache\wheels\1c\c2\b2\f1c7c7d38ed36ef15e596aeef07aba673d7d9650f4134b2ce9
Successfully built pydataxm


In [105]:
datos_AGPE=objetoAPI.request_data(
                                'ListadoAGPE',
                                'Agente', 
                                dt.date(2001, 1, 1), 
                                dt.date(2002, 1, 31))
datos_AGPE

,Id,Values_AgpeID,Values_Name,Values_CompanyCode,Values_ComZone,Values_VoltageLevel,Values_EnerSource,Values_InstalledCapacity,Date
0,Agente,Agp00001,PROMOTORA RD LTDA,CMMC,CARIBE MAR,2,RAD SOLAR,9.000000,2023-03-27
1,Agente,Agp00002,ORGANIZACION TERPEL S A,EPSC,VALLE DEL CAUCA,2,GAS,16.011109,2023-03-27
2,Agente,Agp00003,ANDREA BARRIOS,CNSC,NORTE DE SANTANDER,1,RAD SOLAR,5.000000,2023-03-27
3,Agente,Agp00004,JUAN JOSE SANTA VALLA,CMMC,CARIBE MAR,2,RAD SOLAR,10.000000,2023-03-27
4,Agente,Agp00005,LUIS ALFONSO VILLAMIL VARGAS,CSSC,CARIBE SOL,1,RAD SOLAR,3.000000,2023-03-27
...,...,...,...,...,...,...,...,...,...
5140,Agente,Grp0598,AGPE INVERSIONES SANTA FE SEDE II,GNCC,CARIBE MAR,1,RAD SOLAR,15.000000,2023-03-27
5141,Agente,Grp0601,AGPE GATO DUMAS BOGOTA,BIAC,BOGOTA - CUNDINAMARCA,1,RAD SOLAR,44.800000,2023-03-27
5142,Agente,Unt0499,AGPE - ECOPETROL LA HORMIGA,GECG,BAJO PUTUMAYO,3,GAS,1000.000000,2023-03-27
5143,Agente,Unt0514,AGPE ENTREPALMAS,EPSG,META,2,BIOGAS,500.000000,2023-03-27


In [106]:
compra_AGPE=objetoAPI.request_data(
                                'ExcedenteAGPE',
                                'Agente', 
                                dt.date(2021, 1, 1), 
                                dt.date(2021, 1, 31))


,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Agente,CASC,NaN,NaN,NaN,NaN,NaN,NaN,3.24,26.47,...,6.74,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
1,Agente,CEOC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
2,Agente,CETC,NaN,NaN,NaN,NaN,NaN,NaN,0.01,3.17,...,26.38,14.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
3,Agente,CHCC,0.01,NaN,NaN,NaN,NaN,NaN,3.14,33.55,...,64.75,25.06,4.03,0.02,0.03,0.05,0.04,0.05,0.05,2021-01-01
4,Agente,CMMC,NaN,NaN,NaN,0.00,NaN,NaN,1.25,57.51,...,137.50,51.09,2.01,0.00,0.01,NaN,NaN,NaN,NaN,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,Agente,ENIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-31
472,Agente,EPMC,3.08,5.29,5.45,3.78,3.81,4.11,5.71,41.58,...,1172.35,469.43,72.42,6.70,4.10,4.12,4.17,4.82,4.50,2021-01-31
473,Agente,ESSC,0.20,0.21,0.22,0.22,0.22,0.21,4.70,104.05,...,254.10,107.58,8.06,0.20,0.19,0.18,0.20,0.20,0.21,2021-01-31
474,Agente,HLAC,1.00,0.58,0.58,0.98,0.58,0.98,0.92,33.25,...,47.14,8.09,1.64,0.20,0.58,0.58,0.58,0.98,0.58,2021-01-31


In [111]:
datos_AGPE[datos_AGPE["Values_CompanyCode"]=="HLAC"]["Values_InstalledCapacity"].sum()

2339.425

In [109]:
compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\2271267194.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()
C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\2271267194.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()


Values_Hour01      0.598065
Values_Hour02      0.583226
Values_Hour03      0.583226
Values_Hour04      0.597419
Values_Hour05      0.586452
Values_Hour06      0.598065
Values_Hour07      0.785806
Values_Hour08     12.577097
Values_Hour09     47.088387
Values_Hour10     98.871613
Values_Hour11    150.102903
Values_Hour12    182.498710
Values_Hour13    210.304839
Values_Hour14    178.728710
Values_Hour15    108.965806
Values_Hour16     54.773871
Values_Hour17     15.119032
Values_Hour18      1.233000
Values_Hour19      0.303548
Values_Hour20      0.570968
Values_Hour21      0.576774
Values_Hour22      0.580645
Values_Hour23      0.600645
Values_Hour24      0.582581
dtype: float64